Importing required libraries

In [1]:
import os
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
from tqdm import tqdm
import copy
import time

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torchvision.transforms as transforms
from torchvision import models
from torch import optim, nn

In [2]:
device = 'cpu'

if torch.cuda.is_available() :
    device = 'cuda'

Creating dataset

In [3]:
class ImageDataset(Dataset) :

    def __init__(self, transform) :
        self.root_path = 'PACS/kfold/'

        # Listing the domains
        self.domains = os.listdir(self.root_path)

        # Listing the classes 
        self.classes = os.listdir(self.root_path+'cartoon')

        # Transformations
        self.transforms = transform

        self.images = []
        self.domains_y = []
        self.classes_y = []

        for i_dom, domain in enumerate(self.domains) :
            for i_cla, cla in enumerate(self.classes) :
                for image in os.listdir(self.root_path+domain+'/'+cla) :
                    # Finding image path
                    image_path = self.root_path+domain+'/'+cla+'/'+image
                    img = cv2.imread(image_path)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    self.images.append(img)
                    self.domains_y.append(i_dom)
                    self.classes_y.append(i_cla)

        self.images = np.array(self.images)
        self.domains_y = np.array(self.domains_y)
        self.classes_y = np.array(self.classes_y)

    def __getitem__(self, index) :

        return self.transforms(self.images[index].astype('float')/255), self.domains_y[index], self.classes_y[index]

    def __len__(self) :
        return len(self.images)

Defining transforms

In [4]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize(50),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [5]:
dataset = ImageDataset(transform=transform)

In [6]:
# Train and test split
train_dataset, val_dataset, test_dataset = random_split(dataset, [6000, 1000, 2991])

In [7]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=4, shuffle=True)

In [8]:
dataloaders = {
    'train' : train_dataloader,
    'val' : val_dataloader,
    'test' : test_dataloader
}

dataset_sizes = {
    'train' : 6000,
    'val': 1000,
    'test' : 2991
}

In [9]:
def train_model(model, criterion, optimizer, epochs=1):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 0.0
    best_acc = 0
    
    for ep in range(epochs):
        print(f"Epoch {ep}/{epochs-1}")
        print("-"*10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
                
            for images, domain, labels in dataloaders[phase]:
                images = images.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(images.float())
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    
                running_loss += loss.item() * images.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print(f"{phase} Loss:{epoch_loss:.4f} Acc:{epoch_acc:.4f}")
            
            if phase == 'val':
                if ep == 0:
                    best_loss = epoch_loss
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                else:
                    if epoch_loss < best_loss:
                        best_loss = epoch_loss
                        best_acc = epoch_acc
                        best_model_wts = copy.deepcopy(model.state_dict())
            
        print()
        
    time_elapsed = time.time() - since
    
    print(f'Training complete in {time_elapsed // 60}m {time_elapsed % 60}s')
    print(f'Best val loss: {best_loss:.4f}')
    print(f'Best acc: {best_acc}')
    
    model.load_state_dict(best_model_wts)
    
    return model

In [10]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(3*50*50, 2048),
    nn.ReLU(),
    nn.Linear(2048, 2048),
    nn.ReLU(),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Linear(1024, 256),
    nn.ReLU(),
    nn.Linear(256, 7)
)

# To GPU
model = model.to(device)

# Defining loss function
criterion = nn.CrossEntropyLoss()

# Defining optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [11]:
model = train_model(model, criterion, optimizer, 10)

Epoch 0/9
----------
train Loss:1.7693 Acc:0.2747
val Loss:1.6881 Acc:0.3360

Epoch 1/9
----------
train Loss:1.5790 Acc:0.3777
val Loss:1.6337 Acc:0.3410

Epoch 2/9
----------
train Loss:1.4758 Acc:0.4158
val Loss:1.5441 Acc:0.3840

Epoch 3/9
----------
train Loss:1.3818 Acc:0.4645
val Loss:1.5185 Acc:0.4240

Epoch 4/9
----------
train Loss:1.3040 Acc:0.4978
val Loss:1.6675 Acc:0.3900

Epoch 5/9
----------
train Loss:1.2182 Acc:0.5247
val Loss:1.6428 Acc:0.4170

Epoch 6/9
----------
train Loss:1.1150 Acc:0.5713
val Loss:1.7298 Acc:0.4070

Epoch 7/9
----------
train Loss:1.0455 Acc:0.5990
val Loss:1.7266 Acc:0.4210

Epoch 8/9
----------
train Loss:0.9503 Acc:0.6327
val Loss:1.8248 Acc:0.4020

Epoch 9/9
----------
train Loss:0.9002 Acc:0.6532
val Loss:1.9427 Acc:0.4140

Training complete in 30.0m 12.148520231246948s
Best val loss: 1.5185
Best acc: 0.424
